In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Read recipe inputs
new_inputs = dataiku.Dataset("new_inputs")
new_inputs_df = new_inputs.get_dataframe()

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

inputs_located_df = new_inputs_df # For this sample code, simply copy input to output


# Write recipe outputs
inputs_located = dataiku.Dataset("inputs_located")
inputs_located.write_with_schema(inputs_located_df)